In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
import pandas as pd

from tqdm import tqdm
import re
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight


import math

from sklearn.metrics import f1_score, accuracy_score, classification_report

import matplotlib.pyplot as plt
from tqdm import tqdm
import os

!pip install camel_tools

from camel_tools.tokenizers.word import simple_word_tokenize

os.environ['CAMELTOOLS_DATA'] = '/content/drive/MyDrive/camel_tools'

In [55]:
!pip install Levenshtein
import Levenshtein
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.analyzer import Analyzer
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.disambig.bert import BERTUnfactoredDisambiguator
from pathlib import Path
S31_DB_PATH = Path('/content/drive/My Drive/capstone_data/disambig_db/calima-msa-s31.db')
S31_DB = MorphologyDB(S31_DB_PATH, 'a')
S31_AN = Analyzer(S31_DB, 'NOAN_ALL', cache_size=100000)
bert_disambig = BERTUnfactoredDisambiguator.pretrained('msa', top=1000, pretrained_cache = False)
bert_disambig._analyzer = S31_AN

Some weights of the model checkpoint at /content/drive/MyDrive/camel_tools/data/disambig_bert_unfactored/msa were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Objective
All Wordwise Techniques and Vocabularies. Code so that we can automatically generate results.

As well, produce summary statistics on the vocabularies: MLE, FREQ, SAMERLEX, etc.

### Expected Training and Testing Data:
Punctuation tokenized, all tokens that are either all punctuation or digits removed.

In [56]:
## train data
import numpy as np
words_train = pd.read_csv('/content/drive/My Drive/capstone_data/bashar_data/capstone_data/splits/train_wordwise_clean.csv')
words_dev = pd.read_csv('/content/drive/My Drive/capstone_data/bashar_data/capstone_data/splits/dev_wordwise_clean.csv')
words_test = pd.read_csv('/content/drive/My Drive/capstone_data/bashar_data/capstone_data/splits/test_wordwise_clean.csv')

## testing data in fragments
frag_train = pd.read_csv('/content/drive/My Drive/capstone_data/bashar_data/capstone_data/splits/all_train_aligned.csv')
frag_dev = pd.read_csv('/content/drive/My Drive/capstone_data/bashar_data/capstone_data/splits/all_dev_aligned.csv')
frag_test = pd.read_csv('/content/drive/My Drive/capstone_data/bashar_data/capstone_data/splits/all_test_aligned.csv')


frag_train = frag_train[frag_train.apply(lambda x: type(x['0']) == str, axis = 1)]
frag_dev = frag_dev[frag_dev.apply(lambda x: type(x['0']) == str, axis = 1)]
frag_test = frag_test[frag_test.apply(lambda x: type(x['0']) == str, axis = 1)]

### Acquire BERT Decisions from BERT Notebook

In [57]:
import pickle

### save wordwise decisions
with open('/content/drive/My Drive/capstone_data/final_result/res_wordwise_decisions.pkl', 'rb') as f:
  res_wordwise_decisions = pickle.load(f)

### Acquire FREQ vocabulary

In [58]:
import pickle
with open('/content/drive/My Drive/capstone_data/bashar_data/capstone_data/splits/freq_token_db.pkl', 'rb') as f:
  freq_backoff = pickle.load(f)
with open('/content/drive/My Drive/capstone_data/disambig_db/all_levels_freq_binning_cumulative.pkl', 'rb') as f:
  words_and_levels_freq_binning_cum = pickle.load(f)

alt_backoff = words_and_levels_freq_binning_cum[10000][0]

### Acquire MLE vocabulary

In [59]:
def get_mle_counts_aligned(words, levels):
  dict_levels = {}
  for word, level in zip(words, levels):
      try:
          #assume every entry of dict_levels : {3: int, 4: int, 5: int}
          dict_levels[word][level] += 1
      except:
          dict_levels[word] = {3: 0, 4: 0, 5: 0}
          dict_levels[word][level] += 1
  return dict_levels






### Distributions?

In [60]:
allcounts = get_mle_counts_aligned(words_train['Word'], words_train['Label'])

In [61]:
def max_frequency_strategy(dict_levels, confidence = 0):
  print(confidence)
  dict_levels_max = {}
  no = 0
  for token in dict_levels.keys():
    cd = max(dict_levels[token].values())/sum(dict_levels[token].values())
    if cd >= confidence:
      dict_levels_max[token] = max(dict_levels[token].items(), key = lambda x: x[1])[0]
    else:
      no += 1
  print(no)
  return dict_levels_max

def mle_training_pipeline_aligned(data, strategy, confidence = 0):
  counts = get_mle_counts_aligned(data['Word'], data['Label'])
  return strategy(counts, confidence = confidence)


In [62]:
### look at min number of things - cross the confidence intervals and the threshold appearance minima (1s and 2s, 3s) (cross this)
### look at para level disambig - add to method as tuning for the lexicon.
### send note to

### do as tuning
### Error Analysis
### select a sample of a hundred cases (fragment) - word levels might be lil bit more (report on percentages, and which errors were consequential - inconsequential)
### Automatic classification: look at who caused the error



mle_0 = mle_training_pipeline_aligned(words_train, max_frequency_strategy)
mle_85 = mle_training_pipeline_aligned(words_train, max_frequency_strategy, confidence = 0.80)
mle_90 = mle_training_pipeline_aligned(words_train, max_frequency_strategy, confidence = 0.90)
mle_95 = mle_training_pipeline_aligned(words_train, max_frequency_strategy, confidence = 0.95)
mle_99 = mle_training_pipeline_aligned(words_train, max_frequency_strategy, confidence = 0.99)

0
0
0.8
529
0.9
679
0.95
762
0.99
872


In [63]:
len(mle_99)

31052

### Acquire Lexicon

In [64]:
# configure lexicon
import pickle
with open('/content/drive/My Drive/capstone_data/disambig_db/quick_lemma_lookup.pkl', 'rb') as f:
    lemma_db = pickle.load(f)


def sort_score(list_of_analyses):
  list_of_analyses.sort(key = lambda x: x.score, reverse = True)
  highest_score = list_of_analyses[0].score
  analyses_with_equal_score = [x for x in list_of_analyses
                                if x.score == highest_score]
  return analyses_with_equal_score


def sort_lexlogprob(list_of_analyses):
  list_of_analyses.sort(key = lambda x: x.analysis['lex_logprob'], reverse=True)
  highest_prob = list_of_analyses[0].analysis['lex_logprob']
  analyses_with_equal_prob = [x for x in list_of_analyses
                                if x.analysis['lex_logprob'] == highest_prob]
  return analyses_with_equal_prob

def default_level_oov(word, level, analysis):
  return level

def sort_level(list_of_analyses):
  list_of_analyses.sort(key = lambda x: get_rl_single(x.analysis, 9999))
  lowest_rl = get_rl_single(list_of_analyses[0].analysis, 9999)
  analyses_with_equal_level = [x for x in list_of_analyses
                                    if get_rl_single(x.analysis, 9999) == lowest_rl]
  return analyses_with_equal_level

def score_then_level_then_llp(list_of_analyses):
  list_of_analyses = sort_score(list_of_analyses)
  if len(list_of_analyses) == 1:
    return list_of_analyses[0].analysis

  list_of_analyses = sort_level(list_of_analyses)
  if len(list_of_analyses) == 1:
    return list_of_analyses[0].analysis

  list_of_analyses = sort_lexlogprob(list_of_analyses)
  return list_of_analyses[0].analysis







### Define how to get RL based on each vocabulary

In [65]:
def get_rl_0(token, analyses, oov_level = 0):
  return oov_level

def get_rl_3(token, analyses, oov_level = 3):
  return oov_level

def get_rl_4(token, analyses, oov_level = 4):
  return oov_level

def get_rl_5(token, analyses, oov_level = 5):
  return oov_level

def get_rl_mle(token, analyses, oov_level = 0):
    try:
        return mle_0[token]
    except:
        return oov_level

def get_rl_mle_85(token, analyses, oov_level = 0):
    try:
        return mle_85[token]
    except:
        return oov_level

def get_rl_mle_90(token, analyses, oov_level = 0):
    try:
        return mle_90[token]
    except:
        return oov_level

def get_rl_mle_95(token, analyses, oov_level = 0):
    try:
        return mle_95[token]
    except:
        return oov_level

def get_rl_mle_99(token, analyses, oov_level = 0):
    try:
        return mle_99[token]
    except:
        return oov_level

def get_rl_alt_freq(token, analyses, oov_level = 0):
    try:
        return alt_backoff[token]
    except:
        return oov_level

def get_rl_freq(token, analyses, oov_level = 0):
    try:
        return freq_backoff[token]
    except:
        return oov_level

def get_rl_single(analysis, oov_level = 0):
  lex = analysis['lex']
  pos = analysis['pos']
  if pos == 'noun_prop':
    return 3
  result = lemma_db.get(lex)
  if result:
    if len(result) == 1:
      rl = result[0]['level']
    else:
      most_similar_element = None
      max_similarity = -1

      for element in result:
          similarity = Levenshtein.ratio(pos, element['pos'])
          if similarity > max_similarity:
              max_similarity = similarity
              most_similar_element = element

      rl = most_similar_element['level']
    return rl
  else:
    return oov_level


def get_rl_lexicon(token, analyses, oov_level = 0):
  analysis = score_then_level_then_llp(analyses)

  return get_rl_single(analysis)


In [66]:
def levels_pipeline(fragment, decision_1, decision_2, requires_disambig = False):
  tokens = [t.split('#')[0] for t in fragment.split(' ')]
  gt = [t.split('#')[1] for t in fragment.split(' ')]

  if requires_disambig:
    analyses = [token.analyses for token in bert_disambig.disambiguate(tokens)]
  else:
    analyses = gt

  # decision round 1:
  levels = [decision_1(token, analysis) for token, analysis in zip(tokens, analyses)]

  # decision round 2:
  levels = [l if l != 0 else decision_2(t, a) for l, t, a in zip(levels, tokens, analyses)]

  return {
      'levels': levels,
      'gts': gt,
  }




In [67]:
### New Approach: calculate decisions first, then hierarchy. Saves time!
pure_0 = [levels_pipeline(f, get_rl_0, get_rl_0) for f in frag_dev['0']]


pure_3 = [levels_pipeline(f, get_rl_3, get_rl_3) for f in frag_dev['0']]
pure_4 = [levels_pipeline(f, get_rl_4, get_rl_4) for f in frag_dev['0']]
pure_5 = [levels_pipeline(f, get_rl_5, get_rl_5) for f in frag_dev['0']]

pure_freq = [levels_pipeline(f, get_rl_freq, get_rl_0) for f in frag_dev['0']]
pure_alt_freq = [levels_pipeline(f, get_rl_alt_freq, get_rl_0) for f in frag_dev['0']]
pure_mle = [levels_pipeline(f, get_rl_mle, get_rl_0) for f in frag_dev['0']]
pure_mle_85 = [levels_pipeline(f, get_rl_mle_85, get_rl_0) for f in frag_dev['0']]
pure_mle_90 = [levels_pipeline(f, get_rl_mle_90, get_rl_0) for f in frag_dev['0']]
pure_mle_95 = [levels_pipeline(f, get_rl_mle_95, get_rl_0) for f in frag_dev['0']]
pure_mle_99 = [levels_pipeline(f, get_rl_mle_99, get_rl_0) for f in frag_dev['0']]


pure_lexicon = [levels_pipeline(f, get_rl_lexicon, get_rl_0, requires_disambig = True) for f in frag_dev['0']]


def level_keep0(l):
  if l > 0:
    if l < 3:
      return 3
    else:
      return l
  else:
    return 0

## Lexicon might level 1 or 2.



#### We need to preprocess BERT outputs, as they are padded to 20.


In [68]:
#### Check preformance when padded to 30? (only lose around 150 tokens)
bert_padded_decisions = []

for x, y in zip(pure_alt_freq, res_wordwise_decisions[0]):
  y = y + [0 for x in range(len(x['levels']) - len(y))]
  bert_padded_decisions.append(y)

In [69]:
np.concatenate(bert_padded_decisions).shape

(22075,)

#### Decisions setup

In [70]:

decisions = {
    'pure_0': np.concatenate([e['levels'] for e in pure_0]),
    'pure_3': np.concatenate([e['levels'] for e in pure_3]),
    'pure_4': np.concatenate([e['levels'] for e in pure_4]),
    'pure_5': np.concatenate([e['levels'] for e in pure_5]),
    'pure_freq': np.concatenate([e['levels'] for e in pure_freq]),
    'pure_alt_freq': np.concatenate([e['levels'] for e in pure_alt_freq]),
    'pure_mle': np.concatenate([e['levels'] for e in pure_mle]),
    'pure_mle_85': np.concatenate([e['levels'] for e in pure_mle_85]),

    'pure_mle_90': np.concatenate([e['levels'] for e in pure_mle_90]),

    'pure_mle_95': np.concatenate([e['levels'] for e in pure_mle_95]),

    'pure_mle_99': np.concatenate([e['levels'] for e in pure_mle_99]),

    'pure_lexicon': [level_keep0(l) for l in np.concatenate([e['levels'] for e in pure_lexicon])],
    'pure_bert': [x+3 for x in np.concatenate(bert_padded_decisions)]
}


In [71]:
final_gt = np.concatenate([e['gts'] for e in pure_freq])

In [72]:
pd.Series(decisions['pure_mle']).value_counts()

3    16195
0     4625
4      914
5      341
Name: count, dtype: int64

In [73]:
pd.Series(decisions['pure_lexicon']).value_counts()

3    19053
4     1840
5      798
0      384
Name: count, dtype: int64

In [74]:
#len(res_wordwise_decisions[0])
len(pure_alt_freq)

print(len(pure_alt_freq[2326]['levels']), len(res_wordwise_decisions[0][2326]))

diffs = 0
for i, x, y in zip(range(len(pure_alt_freq)), pure_alt_freq, res_wordwise_decisions[0]):

  if len(x['levels']) != len(y):
    print(i)

8 8
13
37
39
80
125
170
171
178
180
230
299
482
1026
1045
1053
1072
1109
1110
1252
1294
1344
1387
1603
1651
1932
1935
1937
1964
1965
1980
1985
1988
2018
2047
2054
2499
2527
2532
2548
2551
2555
2558
2645


In [75]:
diffs

0

In [76]:
len(decisions['pure_alt_freq'])

22075

### New faster experiments

In [77]:
def comb_experiment_pipeline(decision_1, decision_2 = decisions['pure_0'], decision_final = decisions['pure_0']):
  final_result = []
  ### DECISION 2
  for i, x in enumerate(decision_1):
    if x == 0:
      final_result.append(decision_2[i])
    else:
      final_result.append(x)

  ### Final decision
  for i, x in enumerate(final_result):
    if x == 0:
      final_result[i] = decision_final[i]

  return final_result

exps = [
    comb_experiment_pipeline(decisions['pure_3']),
    comb_experiment_pipeline(decisions['pure_4']),
    comb_experiment_pipeline(decisions['pure_5']),
    comb_experiment_pipeline(decisions['pure_freq']),
    comb_experiment_pipeline(decisions['pure_alt_freq']),
    comb_experiment_pipeline(decisions['pure_bert']),
    comb_experiment_pipeline(decisions['pure_mle'], decision_final = decisions['pure_3']),
    comb_experiment_pipeline(decisions['pure_mle'], decision_final = decisions['pure_4']),
    comb_experiment_pipeline(decisions['pure_mle'], decision_final = decisions['pure_5']),
    comb_experiment_pipeline(decisions['pure_mle'], decision_final = decisions['pure_freq']),
    comb_experiment_pipeline(decisions['pure_mle'], decision_final = decisions['pure_alt_freq']),
    comb_experiment_pipeline(decisions['pure_mle'], decision_final = decisions['pure_bert']),
    comb_experiment_pipeline(decisions['pure_mle'], decision_2 = decisions['pure_lexicon'], decision_final = decisions['pure_3']),
    comb_experiment_pipeline(decisions['pure_mle'], decision_2 = decisions['pure_lexicon'], decision_final = decisions['pure_4']),
    comb_experiment_pipeline(decisions['pure_mle'], decision_2 = decisions['pure_lexicon'], decision_final = decisions['pure_5']),
    comb_experiment_pipeline(decisions['pure_mle'], decision_2 = decisions['pure_lexicon'], decision_final = decisions['pure_freq']),
    comb_experiment_pipeline(decisions['pure_mle'], decision_2 = decisions['pure_lexicon'], decision_final = decisions['pure_alt_freq']),
    comb_experiment_pipeline(decisions['pure_mle'], decision_2 = decisions['pure_lexicon'], decision_final = decisions['pure_bert']),
    comb_experiment_pipeline(decisions['pure_lexicon'], decision_final = decisions['pure_3']),
    comb_experiment_pipeline(decisions['pure_lexicon'], decision_final = decisions['pure_4']),
    comb_experiment_pipeline(decisions['pure_lexicon'], decision_final = decisions['pure_5']),
    comb_experiment_pipeline(decisions['pure_lexicon'], decision_final = decisions['pure_freq']),
    comb_experiment_pipeline(decisions['pure_lexicon'], decision_final = decisions['pure_alt_freq']),
    comb_experiment_pipeline(decisions['pure_lexicon'], decision_final = decisions['pure_bert']),
    comb_experiment_pipeline(decisions['pure_lexicon'], decision_2 = decisions['pure_mle'], decision_final = decisions['pure_3']),
    comb_experiment_pipeline(decisions['pure_lexicon'], decision_2 = decisions['pure_mle'], decision_final = decisions['pure_4']),
    comb_experiment_pipeline(decisions['pure_lexicon'], decision_2 = decisions['pure_mle'], decision_final = decisions['pure_5']),
    comb_experiment_pipeline(decisions['pure_lexicon'], decision_2 = decisions['pure_mle'], decision_final = decisions['pure_freq']),
    comb_experiment_pipeline(decisions['pure_lexicon'], decision_2 = decisions['pure_mle'], decision_final = decisions['pure_alt_freq']),
    comb_experiment_pipeline(decisions['pure_lexicon'], decision_2 = decisions['pure_mle'], decision_final = decisions['pure_bert'])
]

new_exps = [
    comb_experiment_pipeline(decisions['pure_mle_85'], decision_2 = decisions['pure_lexicon'], decision_final = decisions['pure_bert']),
    comb_experiment_pipeline(decisions['pure_mle_90'], decision_2 = decisions['pure_lexicon'], decision_final = decisions['pure_bert']),
    comb_experiment_pipeline(decisions['pure_mle_95'], decision_2 = decisions['pure_lexicon'], decision_final = decisions['pure_bert']),
    comb_experiment_pipeline(decisions['pure_mle_99'], decision_2 = decisions['pure_lexicon'], decision_final = decisions['pure_bert'])
]

### Save the results

In [78]:
with open('/content/drive/My Drive/capstone_data/final_result/wordwise.pkl', 'wb') as f:
  pickle.dump(exps, f)

In [79]:
with open('/content/drive/My Drive/capstone_data/final_result/wordwise.pkl', 'rb') as f:
  exps = pickle.load(f)

In [80]:
len(exps[28])

22075

In [81]:
final_gt = final_gt.astype(int)
def results_to_csv(result_arr):
  all_rows = []
  for resu in result_arr:
    inv_report = classification_report(final_gt, resu, output_dict = True)

    arr_inv = np.concatenate([[inv_report[x]['f1-score'],
            inv_report[x]['precision'],
            inv_report[x]['recall'],] for x in ['3', '4', '5']])
    arr_inv = np.append(arr_inv, inv_report['accuracy'])
    arr_inv = np.append(arr_inv, inv_report['macro avg']['f1-score'])

    all_rows.append(arr_inv)

  return all_rows





In [82]:
all_rows = results_to_csv(exps)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [83]:
all_rows_new = results_to_csv(new_exps)

In [84]:
df_results_old = pd.DataFrame(all_rows, columns = ['f1_3','3_prec','3_recall','f1_4','4_prec','4_recall','f1_5','5_prec','5_recall','accuracy','f1_macro'])

In [85]:
df_results = pd.DataFrame(all_rows_new, columns = ['f1_3','3_prec','3_recall','f1_4','4_prec','4_recall','f1_5','5_prec','5_recall','accuracy','f1_macro'])

In [86]:
df_results_old

,f1_3,3_prec,3_recall,f1_4,4_prec,4_recall,f1_5,5_prec,5_recall,accuracy,f1_macro
0,0.927878,0.865459,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.865459,0.309293
1,0.000000,0.000000,0.000000,0.163248,0.088879,1.000000,0.000000,0.000000,0.000000,0.088879,0.054416
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.087337,0.045663,1.000000,0.045663,0.029112
3,0.841670,0.941462,0.761005,0.207881,0.165907,0.278287,0.285514,0.186830,0.605159,0.710985,0.333766
4,0.929520,0.884067,0.979901,0.214610,0.563991,0.132518,0.147445,0.279006,0.100198,0.864417,0.322894
5,0.965135,0.960259,0.970060,0.679477,0.713165,0.648828,0.592593,0.597980,0.587302,0.924032,0.745735
6,0.951234,0.912056,0.993928,0.575104,0.904814,0.421509,0.412157,0.815249,0.275794,0.910260,0.646165
7,0.899377,0.980179,0.830882,0.459672,0.311248,0.878695,0.412157,0.815249,0.275794,0.809785,0.590402
8,0.899377,0.980179,0.830882,0.575104,0.904814,0.421509,0.298293,0.179420,0.883929,0.796920,0.590925
9,0.915664,0.973416,0.864381,0.510478,0.474398,0.552497,0.397992,0.271254,0.747024,0.831302,0.456033


In [87]:
df_results

,f1_3,3_prec,3_recall,f1_4,4_prec,4_recall,f1_5,5_prec,5_recall,accuracy,f1_macro
0,0.981565,0.975251,0.987961,0.860385,0.892173,0.830785,0.79285,0.843400,0.748016,0.963035,0.878267
1,0.981429,0.975392,0.987543,0.860079,0.890344,0.831804,0.79182,0.839822,0.749008,0.962809,0.877776
2,0.981506,0.975543,0.987543,0.860979,0.890523,0.833333,0.79182,0.839822,0.749008,0.962945,0.878102
3,0.981557,0.975644,0.987543,0.861579,0.890642,0.834353,0.79182,0.839822,0.749008,0.963035,0.878319


In [88]:
df_results.to_csv('/content/drive/My Drive/capstone_data/final_result/wordwise.csv')

### Aggregation Experiments

In [89]:
pure_0 = [levels_pipeline(f, get_rl_0, get_rl_0) for f in frag_dev['0']]


pure_3 = [levels_pipeline(f, get_rl_3, get_rl_3) for f in frag_dev['0']]
pure_4 = [levels_pipeline(f, get_rl_4, get_rl_4) for f in frag_dev['0']]
pure_5 = [levels_pipeline(f, get_rl_5, get_rl_5) for f in frag_dev['0']]

pure_freq = [levels_pipeline(f, get_rl_freq, get_rl_0) for f in frag_dev['0']]
pure_alt_freq = [levels_pipeline(f, get_rl_alt_freq, get_rl_0) for f in frag_dev['0']]
pure_mle = [levels_pipeline(f, get_rl_mle, get_rl_0) for f in frag_dev['0']]


In [90]:
pure_frag_0 = [x['levels'] for x in pure_0]
pure_frag_3 = [x['levels'] for x in pure_3]
pure_frag_4 = [x['levels'] for x in pure_4]
pure_frag_5 = [x['levels'] for x in pure_5]

pure_frag_freq = [x['levels'] for x in pure_freq]
pure_frag_alt_freq = [x['levels'] for x in pure_alt_freq]
pure_frag_mle = [x['levels'] for x in pure_mle]
pure_frag_lexicon = [[l if l > 3 or l == 0 else 3 for l in x['levels']] for x in pure_lexicon]
pure_frag_bert = [[w+3 for w in frag] for frag in bert_padded_decisions]

In [91]:
def frag_exps_pipeline(decision_1, decision_2 = pure_frag_0, decision_final = pure_frag_0, frags = frag_dev['0']):
  all_results = []
  for d1, d2, d3, f in zip(decision_1, decision_2, decision_final, frags):
    toks = [t.split('#')[0] for t in f.split(' ')]
    gts = [int(t.split('#')[1]) for t in f.split(' ')]
    gold_level = max(gts)

    words_5 = [t for t, l in zip(toks, gts) if l == 5]
    words_4 = [t for t, l in zip(toks, gts) if l == 4]
    words_3 = [t for t, l in zip(toks, gts) if l == 3]

    ### predictive process
    ## d2

    round = ['mle' if l != 0 else 0 for l in d1]

    decision = [dec if dec != 0 else alt for dec, alt in zip(d1, d2)]

    round = ['lex' if r == 0 and d != 0 else r for r, d in zip(round, decision)]

    decision = [dec if dec != 0 else alt for dec, alt in zip(decision, d3)]

    round = ['bert' if r == 0 and d != 0 else r for r, d in zip(round, decision)]

    pred = max(decision)


    misidentified_5 = ["{}/{}/{}".format(t, d, r) for t, g, d, r in zip(toks, gts, decision, round) if (d != g and g == 5)]
    misidentified_4 = ["{}/{}/{}".format(t, d, r) for t, g, d, r in zip(toks, gts, decision, round) if (d != g and g == 4)]
    misidentified_3 = ["{}/{}/{}".format(t, d, r) for t, g, d, r in zip(toks, gts, decision, round) if (d != g and g == 3)]

    all_errors = [' '.join(["{}->{}->{}".format(g, r, d) for t, g, d, r in zip(toks, gts, decision, round) if (d != g and g == x)]) for x in [3,4,5]]

    all_results.append([
        gold_level,
        pred
    ])
  return all_results


In [92]:
exps = [
    frag_exps_pipeline(pure_frag_3),
    frag_exps_pipeline(pure_frag_4),
    frag_exps_pipeline(pure_frag_5),
    frag_exps_pipeline(pure_frag_freq),
    frag_exps_pipeline(pure_frag_alt_freq),
    frag_exps_pipeline(pure_frag_bert),
    frag_exps_pipeline(pure_frag_mle, decision_final = pure_frag_3),
    frag_exps_pipeline(pure_frag_mle, decision_final = pure_frag_4),
    frag_exps_pipeline(pure_frag_mle, decision_final = pure_frag_5),
    frag_exps_pipeline(pure_frag_mle, decision_final = pure_frag_freq),
    frag_exps_pipeline(pure_frag_mle, decision_final = pure_frag_alt_freq),
    frag_exps_pipeline(pure_frag_mle, decision_final = pure_frag_bert),
    frag_exps_pipeline(pure_frag_mle, decision_2 = pure_frag_lexicon, decision_final = pure_frag_3),
    frag_exps_pipeline(pure_frag_mle, decision_2 = pure_frag_lexicon, decision_final = pure_frag_4),
    frag_exps_pipeline(pure_frag_mle, decision_2 = pure_frag_lexicon, decision_final = pure_frag_5),
    frag_exps_pipeline(pure_frag_mle, decision_2 = pure_frag_lexicon, decision_final = pure_frag_freq),
    frag_exps_pipeline(pure_frag_mle, decision_2 = pure_frag_lexicon, decision_final = pure_frag_alt_freq),
    frag_exps_pipeline(pure_frag_mle, decision_2 = pure_frag_lexicon, decision_final = pure_frag_bert),
    frag_exps_pipeline(pure_frag_lexicon, decision_final = pure_frag_3),
    frag_exps_pipeline(pure_frag_lexicon, decision_final = pure_frag_4),
    frag_exps_pipeline(pure_frag_lexicon, decision_final = pure_frag_5),
    frag_exps_pipeline(pure_frag_lexicon, decision_final = pure_frag_freq),
    frag_exps_pipeline(pure_frag_lexicon, decision_final = pure_frag_alt_freq),
    frag_exps_pipeline(pure_frag_lexicon, decision_final = pure_frag_bert),
    frag_exps_pipeline(pure_frag_lexicon, decision_2 = pure_frag_mle, decision_final = pure_frag_3),
    frag_exps_pipeline(pure_frag_lexicon, decision_2 = pure_frag_mle, decision_final = pure_frag_4),
    frag_exps_pipeline(pure_frag_lexicon, decision_2 = pure_frag_mle, decision_final = pure_frag_5),
    frag_exps_pipeline(pure_frag_lexicon, decision_2 = pure_frag_mle, decision_final = pure_frag_freq),
    frag_exps_pipeline(pure_frag_lexicon, decision_2 = pure_frag_mle, decision_final = pure_frag_alt_freq),
    frag_exps_pipeline(pure_frag_lexicon, decision_2 = pure_frag_mle, decision_final = pure_frag_bert)
]

In [93]:
def frag_results_to_csv(result_arr):
  all_rows = []
  for resu in result_arr:
    rr = pd.DataFrame(resu, columns = ['gt', 'pred'])

    inv_report = classification_report(rr['gt'], rr['pred'], output_dict = True)

    arr_inv = np.concatenate([[inv_report[x]['f1-score'],
            inv_report[x]['precision'],
            inv_report[x]['recall'],] for x in ['3', '4', '5']])
    arr_inv = np.append(arr_inv, inv_report['accuracy'])
    arr_inv = np.append(arr_inv, inv_report['macro avg']['f1-score'])

    all_rows.append(arr_inv)

  return all_rows

In [98]:
frag_df = pd.DataFrame(frag_results_to_csv(exps))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [99]:
frag_df

,0,1,2,3,4,5,6,7,8,9,10
0,0.597526,0.426052,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.426052,0.199175
1,0.000000,0.000000,0.000000,0.478059,0.314111,1.000000,0.000000,0.000000,0.000000,0.314111,0.159353
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.412493,0.259837,1.000000,0.259837,0.137498
3,0.392298,0.802956,0.259554,0.271420,0.322437,0.234341,0.489734,0.345494,0.840731,0.402646,0.288363
4,0.649358,0.506003,0.906051,0.293251,0.520661,0.204104,0.295351,0.489426,0.211488,0.505088,0.309490
5,0.850693,0.846336,0.855096,0.702023,0.710963,0.693305,0.693904,0.689433,0.698433,0.763569,0.748873
6,0.751395,0.615228,0.964968,0.620993,0.742857,0.533477,0.502317,0.865815,0.353786,0.670624,0.624902
7,0.545866,0.945098,0.383758,0.576205,0.413647,0.949244,0.502317,0.865815,0.353786,0.553596,0.541463
8,0.545866,0.945098,0.383758,0.265568,0.873494,0.156587,0.493746,0.330106,0.979112,0.467096,0.435060
9,0.605166,0.895476,0.457006,0.439532,0.552288,0.365011,0.558991,0.406028,0.896867,0.542402,0.400922


In [100]:
frag_df.to_csv('/content/drive/My Drive/capstone_data/final_result/fwise.csv')